In [39]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets as dt

In [40]:
#method to generate random samples following Normal distribution in d dimensions
def multi_gaussian(dim, num_of_samples):
    cov_matrix = dt.make_spd_matrix(dim) #generate random symmetric, positive definite matrix
    mean = np.random.rand(dim)
    x = np.random.multivariate_normal(mean, cov_matrix, (num_of_samples)) #shape of x is num_of_samples x dim
    return x
# multi_gaussian(5,10)

In [84]:
def discriminant_function(x, mean_vec, cov_matrix, dim, prior_prob):
        if dim > 1: # multivariate gaussian
            cov_inverse = np.linalg.inv(cov_matrix)
            cov_det = np.linalg.det(cov_matrix)
            return (
                    ((- 1/2) * np.square( mahalanobis_dist( x, mean_vec, cov_inverse ))) \
                    - ((dim / 2) * ( np.log(2 * np.pi)) )\
                    - ( (1/2) * np.log(cov_det) ) \
                    + np.log(prior_prob)
            )
        else: # univariate gaussian
#             print("euc_dist",euclidean_dis(x, mean_vec))
#             print(cov_matrix)
            return (
                    -(1/2) * np.square( euclidean_dis(x, mean_vec)) /  cov_matrix \
                    - (1/2) * (np.log(2 * np.pi * cov_matrix))\
                    + np.log(prior_prob)
            )

In [42]:
def euclidean_dis(x, mean_vector):
    cov_inverse = np.identity(x.shape[1])
    return mahalanobis_dist(x, mean_vector, cov_inverse) #

In [85]:
def mahalanobis_dist(x, mean_vector, cov_inverse):
#     print("shape of x in mahalanobis dist", x.shape)
    diff = ( x - mean_vector )
    return np.sqrt( np.dot( np.dot(diff.T, cov_inverse), diff ))

# Section

In [92]:
data = np.genfromtxt('data_dhs_chap2.csv', delimiter=',', skip_header=1)
data

array([[-5.01, -8.12, -3.68,  0.  ],
       [-5.43, -3.48, -3.54,  0.  ],
       [ 1.08, -5.52,  1.66,  0.  ],
       [ 0.86, -3.78, -4.11,  0.  ],
       [-2.67,  0.63,  7.39,  0.  ],
       [ 4.94,  3.29,  2.08,  0.  ],
       [-2.51,  2.09, -2.59,  0.  ],
       [-2.25, -2.13, -6.94,  0.  ],
       [ 5.56,  2.86, -2.26,  0.  ],
       [ 1.03, -3.33,  4.33,  0.  ],
       [-0.91, -0.18, -0.05,  1.  ],
       [ 1.3 , -2.06, -3.53,  1.  ],
       [-7.75, -4.54, -0.95,  1.  ],
       [-5.47,  0.5 ,  3.92,  1.  ],
       [ 6.14,  5.72, -4.85,  1.  ],
       [ 3.6 ,  1.26,  4.36,  1.  ],
       [ 5.37, -4.63, -3.65,  1.  ],
       [ 7.18,  1.46, -6.66,  1.  ],
       [-7.39,  1.17,  6.3 ,  1.  ],
       [-7.5 , -6.32, -0.31,  1.  ],
       [ 5.35,  2.26,  8.13,  2.  ],
       [ 5.12,  3.22, -2.66,  2.  ],
       [-1.34, -5.31, -9.87,  2.  ],
       [ 4.48,  3.42,  5.19,  2.  ],
       [ 7.11,  2.39,  9.21,  2.  ],
       [ 7.17,  4.33, -0.98,  2.  ],
       [ 5.75,  3.97,  6.65,  2.  ],
 

In [79]:
def dichotomizer(X, Y, dim):
    if dim == 1 : # only 1 independent feature
        class1_mean_vec = np.mean( X[:5] )
        class2_mean_vec = np.mean( X[5:10] )
        class1_cov = np.cov( X[:5] )
        class2_cov = np.cov( X[5:10] )
        shape = (1,1)
    else: # more than 1
        class1_mean_vec = np.mean( X[:5], axis=0 )
        class2_mean_vec = np.mean( X[5:10], axis=0 )
        class1_cov = np.cov( X[:5].T )
        class2_cov = np.cov( X[5:10].T )
        shape = (dim,)
    class1_prior_prob = 0.5
    class2_prior_prob = 0.5
    predicted_class = []
    for instance in X:
        instance = instance.reshape( shape )
        g1 = discriminant_function(instance, class1_mean_vec, class1_cov, dim, class1_prior_prob)
        g2 = discriminant_function(instance, class2_mean_vec, class2_cov, dim, class2_prior_prob)
        if g1 > g2 :
            predicted_class.append(0) # class 1
        else :
            predicted_class.append(1) # class 2
    return predicted_class

In [80]:
def empirical_training_error(target_class, predicted_class):
    total_instances = len(predicted_class)
    error = 0 
    for instance in range(total_instances) :
        error += np.abs( target_class[instance] - predicted_class[instance] )
    avg_error_percent = (100 / total_instances) * error
    return avg_error_percent

In [106]:
# print(data)
trg_data =  data[ 5:15 , [0,3] ] 
X = trg_data[:, 0] 
Y = trg_data[:, 1].astype(int)
# print(trg_data)
target_class = []
for i in Y:
    target_class.append(i.item())
# print(target_class)
predicted_class = dichotomizer(X, Y, 1)
print("predicted class labels:", predicted_class)
print("misclassification error: " + str(empirical_training_error(target_class, predicted_class)) + " %")

('predicted class labels:', [1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
misclassification error: 50 %


In [107]:
trg_data =  data[ 5:15 , [0,1,3] ] 
X = trg_data[:, :2] 
Y = trg_data[:, -1].astype(int)
target_class = []
for i in Y:
    target_class.append(i.item())
predicted_class = dichotomizer(X, Y, 2)
print("predicted class labels:", predicted_class)
print("misclassification error: " + str(empirical_training_error(target_class, predicted_class)) + " %")

('predicted class labels:', [0, 1, 1, 0, 0, 1, 0, 1, 1, 1])
misclassification error: 30 %


In [108]:
trg_data =  data[ 5:15 , [0,1,2,3] ] 
X = trg_data[:, :3] 
Y = trg_data[:, 3].astype(int)
target_class = []
for i in Y:
    target_class.append(i.item())
predicted_class = dichotomizer(X, Y, 3)
print("predicted class labels:", predicted_class)
print("misclassification error: " + str(empirical_training_error(target_class, predicted_class)) + " %")

('predicted class labels:', [0, 0, 0, 0, 0, 1, 0, 1, 1, 1])
misclassification error: 10 %


In [121]:
#2(e)
test_data = np.array([
    [1,2,1],
    [5,3,2],
    [0,0,0],
    [1,0,0],
])

class1_samples = data[:10,[0,1,2]]
class2_samples = data[:20,[0,1,2]]
class3_samples = data[:30,[0,1,2]]

class1_mean = np.mean(class1_samples, axis=0)
class2_mean = np.mean(class2_samples, axis=0)
class3_mean = np.mean(class3_samples, axis=0)
# print(class1_mean, class2_mean, class3_mean)
# print(class1_mean.shape)

class1_cov = np.cov(class1_samples.T)
class2_cov = np.cov(class2_samples.T)
class3_cov = np.cov(class3_samples.T)
# print(class1_cov, class2_cov, class3_cov)

predicted_class = []
for x in test_data:
#     print(x.shape)
    distances = []
    distances.append( mahalanobis_dist(x, class1_mean, class1_cov) )
    distances.append( mahalanobis_dist(x, class2_mean, class2_cov) )
    distances.append( mahalanobis_dist(x, class3_mean, class3_cov) )
    print(distances)
    predicted_class.append(distances.index(min(distances)))

print(predicted_class)

[21.275575314961024, 17.90871050977826, 11.219049962629267]
[39.122669886373224, 37.01822619351304, 32.03792965159817]
[9.17501158099911, 6.6910663906900005, 3.9810165992798727]
[12.018736518930258, 10.307336334245278, 1.3902868654848097]
[2, 2, 2, 2]


In [123]:
# 2(f)
dim = 3
class1_prior_prob = 0.8
class2_prior_prob = 0.1
class3_prior_prob = 0.1

# print(class1_mean)

predicted_class = []

for x in test_data:
#     print(x.shape)
#     print(class1_mean.shape)
    values = []
    values.append( np.abs( discriminant_function( x, class1_mean, class1_cov, dim, class1_prior_prob )))
    values.append( np.abs( discriminant_function( x, class2_mean, class2_cov, dim, class2_prior_prob )))
    values.append( np.abs( discriminant_function( x, class3_mean, class3_cov, dim, class3_prior_prob )))
    predicted_class.append(values.index(max(values)))
    print(values)
print(predicted_class)

[3.09118597961416, 5.0042118472888895, 4.82769396465147]
[4.485700025890535, 6.529799857793176, 5.14821405557784]
[2.3010829299405198, 4.270385758480403, 4.272408284456987]
[2.298420276739546, 4.318965947115872, 4.160693425047617]
[1, 1, 2, 1]


# Working with Iris Dataset

Linear Discriminant Function- Case 1- covariance matrices are equal and proportional to Identity matrix

In [114]:
iris_data = np.genfromtxt('iris.csv', delimiter=',')
X = iris_data[:,:4]
print(X.shape)
label_to_nominal = {
    'Iris-setosa' : 0,
    'Iris-versicolor' : 1,
    'Iris-virginica' : 2
}
class1 = np.full((1,50), label_to_nominal['Iris-setosa'], dtype=int) # 
class2 = np.full((1,50), label_to_nominal['Iris-versicolor'], dtype=int) 
class3 = np.full((1,50), label_to_nominal['Iris-virginica'], dtype=int)
Y = np.append(class1, class2)
Y = np.append(Y, class3)

(150, 4)


In [124]:
class1_samples = X[0:50,]
class2_samples = X[50:100,]
class3_samples = X[100:150,]
# print(class1_samples)

iris_class1_mean = np.mean(class1_samples, axis=0)
iris_class2_mean = np.mean(class2_samples, axis=0)
iris_class3_mean = np.mean(class3_samples, axis=0)

iris_class1_cov = np.cov(class1_samples.T)
iris_class2_cov = np.cov(class2_samples.T)
iris_class3_cov = np.cov(class3_samples.T)

# print(class1_cov, class2_cov, class3_cov )
# class1_mean.shape

In [98]:
def quad_discriminant(x, mean_vec, cov_mat, prior_prob):
    cov_inverse = np.linalg.inv(cov_matrix)
    cov_det = np.linalg.det(cov_matrix)
    
    Wi = (-1 / 2) * (cov_inverse)
    wi = np.dot(cov_inverse, mean_vec)
    wio = (-1 / 2) * ( np.dot(np.dot(mean_vec.T, cov_inverse), mean_vec) + np.log(cov_det) ) + np.log(prior_prob) 
    
    g = np.dot(np.dot(x.T, Wi), x) + np.dot(wi.T, x) + wio
    return g

In [55]:
def euclidean_norm(x, mean_vec):
    return np.dot((x - mean_vec).T, (x - mean_vec))

In [56]:
#Case I : all cov matrices are equal and proportional to I(identity matrix)
def case1_linear_discriminant(x, mean_vec, var, prior_prob):
    return (
        -( euclidean_norm(x, mean_vec) / (2 * np.square(var)) ) + np.log(prior_prob)
    )

In [140]:
#Case1: var = avg variance (of diagonal elements)
# print()

diag1 = np.mean(iris_class1_cov.diagonal())
diag2 = np.mean(iris_class2_cov.diagonal())
diag3 = np.mean(iris_class3_cov.diagonal())

var = (diag1+diag2+diag3)/3
predicted_class = []
for x in X:
    g1 = case1_linear_discriminant(x, iris_class1_mean, var, )

0.15201836734693877